In [7]:
!pip install pyarrowfs-adlgen2
!pip install azure-identity
!pip install azure-storage-file-datalake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.9/187.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.4 MB/s eta 0:00:00


In [7]:
!pwd

/content


In [33]:
# import tensorflow_decision_forests as tfdf
import ydf
import numpy as np
import pandas as pd
import tensorflow as tf
import tf_keras
import os

import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq
import pyarrowfs_adlgen2 as pa_adl

from dotenv import load_dotenv
from pathlib import Path

from azure.identity import DefaultAzureCredential, ClientSecretCredential
from azure.storage.filedatalake import DataLakeServiceClient


In [9]:
# Retrieve credentials from environment variables
# this is strictly used only in development
# load env variables
env_dir = Path('./').resolve()
load_dotenv(os.path.join(env_dir, '.env'))

True

In [12]:
storage_account_name = os.environ.get("STORAGE_ACCOUNT_NAME")
credential = os.environ.get("STORAGE_ACCOUNT_KEY")
conn_str = os.environ.get("STORAGE_ACCOUNT_CONN_STR")

In [16]:
# cloud
# URL = "abfss://{FOLDER_NAME}@sgppipelinesa.dfs.core.windows.net"
URL = "{FOLDER_NAME}"
GOLD_FOLDER_NAME = "sgppipelinesa-gold"
GOLD_DATA_DIR = os.path.join("{FOLDER_NAME}").replace("\\", "/")
GOLD_DATA_DIR

'{FOLDER_NAME}'

In [13]:
handler = pa_adl.AccountHandler.from_account_name(storage_account_name, credential=credential)
fs = pa.fs.PyFileSystem(handler)

In [48]:
# cloud
train_data_sc_sm_red_table_path = os.path.join(
    GOLD_DATA_DIR.format(
        FOLDER_NAME=GOLD_FOLDER_NAME
    ),
    "train_data_sc_sm_red.parquet"
).replace("\\", "/")
train_data_sc_sm_red_table = pq.read_table(train_data_sc_sm_red_table_path, filesystem=fs)

In [49]:
# cloud
val_data_sc_sm_red_table_path = os.path.join(
    GOLD_DATA_DIR.format(
        FOLDER_NAME=GOLD_FOLDER_NAME
    ),
    "val_data_sc_sm_red.parquet"
).replace("\\", "/")
val_data_sc_sm_red_table = pq.read_table(val_data_sc_sm_red_table_path, filesystem=fs)

In [50]:
# cloud
test_data_sc_sm_red_table_path = os.path.join(
    GOLD_DATA_DIR.format(
        FOLDER_NAME=GOLD_FOLDER_NAME
    ),
    "test_data_sc_sm_red.parquet"
).replace("\\", "/")
test_data_sc_sm_red_table = pq.read_table(test_data_sc_sm_red_table_path, filesystem=fs)

In [52]:
# returns the pyarrow table as a dictionary e.g.
# {
#     'f1': [...],
#     'f2': [...],
#     ...
#     'fn': [...],
#     'label': [...]
# }
train_data_sc_sm_red_dict = {
    key: np.array(item).astype(int)
    if key == "label"
    else np.array(item)
    for key, item in train_data_sc_sm_red_table.to_pydict().items()
  }
train_data_sc_sm_red_dict.get("label")[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [53]:
# returns the pyarrow table as a dictionary e.g.
# {
#     'f1': [...],
#     'f2': [...],
#     ...
#     'fn': [...],
#     'label': [...]
# }
val_data_sc_sm_red_dict = {
    key: np.array(item).astype(int)
    if key == "label"
    else np.array(item)
    for key, item in val_data_sc_sm_red_table.to_pydict().items()
  }
val_data_sc_sm_red_dict.get("label")[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [54]:
# returns the pyarrow table as a dictionary e.g.
# {
#     'f1': [...],
#     'f2': [...],
#     ...
#     'fn': [...],
#     'label': [...]
# }
test_data_sc_sm_red_dict = {
    key: np.array(item).astype(int)
    if key == "label"
    else np.array(item)
    for key, item in test_data_sc_sm_red_table.to_pydict().items()
  }
test_data_sc_sm_red_dict.get("label")[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [55]:
len(train_data_sc_sm_red_dict.get("label"))

367776

In [56]:
model = ydf.GradientBoostedTreesLearner(
    label="label",
    include_all_columns=True,
    task=ydf.Task.CLASSIFICATION,

)

In [ ]:
model.train(train_data_sc_sm_red_dict, val_data_sc_sm_red_dict)

Train model on 367776 training examples and 80458 validation examples
